## 2 Ground Control Stations (simple)

This is a first version. The two ground control stations are working in series. 

In [1]:
# Custom modules
from helpers import local2global, kill_processes,clean
from simulators import Simulator, Gazebo, ConfigGazebo, QGC
from plan import Plan
from oracle import Oracle,GCS
from config import Color

kill_processes()

In [2]:
import pymavlink
print(pymavlink.__version__)


2.4.43


In [3]:
from pymavlink import mavutil
mavutil.mavlink.MAV_TYPE_GCS 

6

In [4]:
mavutil.mavlink.MAV_AUTOPILOT_INVALID

8

## Create Plans

In [5]:
offsets = [  # east, north, up, heading
    (5., 5., 0., 90.),
    (10., 0., 0., 45.),
    (-5., -10., 0., 225.),
    (-15., 0., 0., 0.),
    (0., -20., 0., 0.),
]
n_vehicles = len(offsets)
side_lens = (5, 7, 4, 1, 2)  # must agree with n_vehicles
local_paths = [Plan.create_square_path(side_len=side_len, alt=5) for side_len in side_lens]
plans = [Plan.basic(wps=path, wp_margin=0.5) for path in local_paths]

## Visualization Parameters

In [6]:
homes = [offset[:3] for offset in offsets]
global_paths = [local2global(path, home) for path, home in zip(local_paths, homes)]
models = 2*[('iris',Color.BLUE)]+3*[('iris',Color.GREEN)]  # len must agree with n_vehicles
markers = [ConfigGazebo.create_trajectory_from_array(array=global_paths[i],color=models[i][1]) for i in range(n_vehicles)]
Gazebo.plot_3d_interactive(markers=markers,title="Simulation Waypoints",frames=(0.2, 0.2, 0.6),ground=-0.05)

## Choose Simulator

In [7]:

gazebo_config = ConfigGazebo(world_path="simulators/gazebo/worlds/runway.world",models=models,markers = markers)
simulator = Gazebo(offsets=offsets,plans=plans,config=gazebo_config)

simulator = QGC(offsets=offsets, plans=plans, origin=(-35.3633245, 149.1652241, 0, 0))


simulator = Simulator(offsets=offsets, plans=plans)

In [8]:
#simulator._delete_all_qgc_links()

Launch Simulator

In [9]:
sysids = list(range(1,n_vehicles+1))
simulator.launch()
orac = Oracle(sysids)
blue_gcs = GCS(sysids[:2],name='blue 🟦')
green_gcs = GCS(sysids[2:],name = 'green 🟩')

while len(orac.conns):
    blue_gcs.gather_broadcasts()
    green_gcs.gather_broadcasts()
    blue_gcs.save_pos()
    green_gcs.save_pos() 
    for sysid in list(orac.conns.keys()):
        if orac.is_plan_done(sysid):
            orac.remove(sysid)
clean()

🚀 Vehicle 1 launched (PID 2044679)
🚀 Vehicle 1 logic launched (PID 2044760)
🚀 Vehicle 2 launched (PID 2044761)
🚀 Vehicle 2 logic launched (PID 2044874)
🚀 Vehicle 3 launched (PID 2044875)
🚀 Vehicle 3 logic launched (PID 2044990)
🚀 Vehicle 4 launched (PID 2044991)
🚀 Vehicle 4 logic launched (PID 2045118)
🚀 Vehicle 5 launched (PID 2045119)
🚀 Vehicle 5 logic launched (PID 2045234)
ℹ️  Running without visualization.
🔗 UAV logic 1 is connected to Oracle ⚪
🔗 UAV logic 2 is connected to Oracle ⚪
🔗 UAV logic 3 is connected to Oracle ⚪
🔗 UAV logic 4 is connected to Oracle ⚪
🔗 UAV logic 5 is connected to Oracle ⚪
🔗 UAV logic 1 is connected to GCS blue 🟦
🔗 UAV logic 2 is connected to GCS blue 🟦
🔗 UAV logic 3 is connected to GCS green 🟩
🔗 UAV logic 4 is connected to GCS green 🟩
🔗 UAV logic 5 is connected to GCS green 🟩
✅ Vehicle 4 terminated
✅ Vehicle 5 terminated
✅ Vehicle 3 terminated
✅ Vehicle 1 terminated
✅ Vehicle 2 terminated


## Print Positions gathered

In [10]:
import numpy as np
from numpy.typing import NDArray
from typing import List
blue_paths:List[NDArray[np.float64]] = []
for sysid,path in blue_gcs.paths.items():
    blue_paths.append(np.array([x for x in path if x is not None]))

markers = [ConfigGazebo.create_trajectory_from_array(array=path,color=Color.BLUE) for path in blue_paths]
Gazebo.plot_3d_interactive(markers=markers,title="Simulation Waypoints",frames=(0.2, 0.2, 0.6),ground=-0.05)

In [11]:
green_paths = []
for sysid,path in green_gcs.paths.items():
    green_paths.append(np.array([x for x in path if x is not None]))

markers = [ConfigGazebo.create_trajectory_from_array(array=path,color=Color.GREEN) for path in green_paths]
Gazebo.plot_3d_interactive(markers=markers,title="Simulation Waypoints",frames=(0.2, 0.2, 0.6),ground=-0.05)